In [134]:
import paramiko
import os
from getpass import getpass

# Checks to see if the results files already exist.
# if the file does exist then the user has the option to delete it.
# the user can also ignore the warnings and append to the existing file.
flist = ["kraken_results.body", "lsof_results"]
for eachFile in flist:
    if os.path.exists(eachFile):
        # If so ask user to delete file
        user_input = input("The file: "+ eachFile + " already exists. Enter 'y' to delete it:")

        # If yes delete the file
        if user_input == "y":
            os.remove(eachFile)

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host Information
host = "192.168.6.71"
port = 2222
username = "blaise.notter"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

## Task 2: Uploading the Kraken Binary and running it against the remote system

In [135]:
sftp = ssh.open_sftp()

# Set Kraken as a string
script = 'kraken'

# Concat my remote home dir to the script
# Then upload the script
remote_path = '/home/blaise.notter/' + script
sftp.put(script, remote_path)

# Run the Kraken file against the host as root
command = ['sudo -S chmod +x kraken','sudo -S ./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin']

# Loop through the files and download them
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)

    # Input password to run command as sudo, including a newline
    # character that provides 'Enter' as if you were typing it on a keyboard
    stdin.write(thePass + "\n")

    # Get results from stdout
    lines = stdout.readlines()
    # print(lines)

    # Covert the list to a string
    output = ''.join(lines)

    # Header Output
    sepHeader = '' + '### BEGIN ' + eachCMD + '###\n\n'

    # Footer Output
    sepFooter = '' + '### END ' + eachCMD + '###\n\n'

    # Concatenate the header, output, and footer
    cmd_output = sepHeader + output +sepFooter

    # Write to a file
    with open('kraken_results.body', 'a') as f:
        f.write(cmd_output)

    # Print the output for quick refrence
    print(cmd_output)

### BEGIN sudo -S chmod +x kraken###

### END sudo -S chmod +x kraken###


### BEGIN sudo -S ./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin###

INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0005] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0014] Scanning autoruns...                         
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0016] Scanning the filesystem (this can take several minutes)... 

## Task 3: Look for any suspicious PIDs in the Kraken results

In [136]:
# Suspicious process id
pid = "3138"

# Run the lsof command against the discovered pid
command = 'sudo -S lsof -p ' + pid

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)

# Input password to run command as sudo, including a newline
# character that provides 'Enter' as if you were typing it on a keyboard
stdin.write(thePass + "\n")

# Get results from stdout
lines = stdout.readlines()
# print(lines)

# Covert the list to a string
output = ''.join(lines)

# Header Output
sepHeader = '' + '### BEGIN ' + eachCMD + '###\n\n'

# Footer Output
sepFooter = '' + '### END ' + eachCMD + '###\n\n'

# Concatenate the header, output, and footer
cmd_output = sepHeader + output +sepFooter

# Write to a file
with open('lsof_results', 'a') as f:
    f.write(cmd_output)

# Print the output
print(cmd_output)

### BEGIN sudo -S ./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin###

COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME
ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk
ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /
ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls
ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so
ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log

## Task 4: Download any associated suspicious files, binaries, or/and logs

In [137]:
# List of sus files to download from the host
files = ['/home/p.dalton/test-lk/.d.log', '/home/p.dalton/test-lk/d.pid']

# Loop through the files and download them
for eachFile in files:

    # Create a list from the filename
    x = eachFile.split("/")

    # -1 is used to pluck the last element in a list.
    filename = x[-1]

    # Local path to store files
    local_path = "remote_files/" + filename

    sftp.get(eachFile, local_path)

## Task 5: Check for ThugStyle activity in other places

In [138]:
# Commands to be run against the host
# These are files needed for the investigation
# They are moved to a usr owned dir and downloaded below
command = ['sudo -S dir /home/p.dalton/ > /home/blaise.notter/user_home.txt','sudo -S ps aux > /home/blaise.notter/processes.txt','sudo -S cp /var/log/auth.log .', 'sudo -S chmod +r auth.log', 'sudo -S cp /home/p.dalton/aqwerpuqwerj .', 'sudo -S chmod +r aqwerpuqwerj']

# Loop through the files and download them
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)

    # Input password to run command as sudo, including a newline
    # character that provides 'Enter' as if you were typing it on a keyboard
    stdin.write(thePass + "\n")


# List of files to download
files = ['/etc/passwd', '/home/blaise.notter/auth.log','/home/blaise.notter/processes.txt', '/home/blaise.notter/user_home.txt', '/home/blaise.notter/aqwerpuqwerj']

# Loop through the files and download them
for eachFile in files:

    # Create a list from the filename
    x = eachFile.split("/")

    # -1 is used to pluck the last element in a list.
    filename = x[-1]

    # Local path to store files
    local_path = "remote_files/" + filename

    # Pulling the files from the remote host
    sftp.get(eachFile, local_path)

**What is the process name?**
The "go-daemon sample"

**Which user is running the process?**
p.dalton

**Has the user logged in before via SSH?**
Yes

**When did the user login?**
Feb 27 04:44:14

**How long was the login session?**
10 seconds. However, he logs back in few more times shortly after for durations of around 10 seconds.

**What is located in the unauthorized user’s home directory?**
A directory called `test-lk/` and a file named "aqwerpuqwerj". I updated my code after seeing this file to include it in the downloads.


In [139]:
ssh.close()

## Reflection
**Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?**
Based on the information and the evidence discovered in this investigation, I would say the host was possibly hit by the threat actor ThugStyle. The user account matches the format (x.xxxxx), the file discovered in the home directory matched with previous reports ("executables were discovered with a high entropy (random) filename but have only been seen in lowercase letters between 6 and 17 character"), and the process had made refrence to Go with its name. The Kraken scan also showed GoBinaries detected.

**What did you like the most and least about this assignment?**
The assignment was pretty enjoyable, I liked that it simulated a real investigation. I don't know what I didn't like about this assignent. It was just fun.

**What additional questions do you have?**
I was curious to see what the executable was that I found in the user's home directory. I am too afraid to run it and see what it does, well also it's just smart move to run unknown executables.